## Project Scenario

#### Imagine you have just been hired by a US Venture Capital firm as a data scientist.

 The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

 Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price. For now they are mainly interested in a macro-view of Canada’s crop farming industry, and how it relates to the relative value of the Canadian and US dollars.

We need to work with 4 datasets:

Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv

Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv

Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv

Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv

## Load all the libraries I need for the project

In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


character(0)

In [2]:
library(RSQLite)


### Create connection

In [3]:
connect <- dbConnect(RSQLite::SQLite(),"FinalDB.sqlite")

### Create tables

In [4]:
crop_data <- dbExecute(connect, "CREATE TABLE CROP_DATA (
                                      CD_ID INTEGER NOT NULL,
                                      YEAR DATE NOT NULL,
                                      CROP_TYPE VARCHAR(20) NOT NULL,
                                      GEO VARCHAR(20) NOT NULL, 
                                      SEEDED_AREA INTEGER NOT NULL,
                                      HARVESTED_AREA INTEGER NOT NULL,
                                      PRODUCTION INTEGER NOT NULL,
                                      AVG_YIELD INTEGER NOT NULL,
                                      PRIMARY KEY (CD_ID)
                                      )")
farm_prices <- dbExecute(connect, "CREATE TABLE FARM_PRICES (
                                        CD_ID INTEGER NOT NULL,
                                        DATE DATE NOT NULL,
                                        CROP_TYPE VARCHAR(20) NOT NULL,
                                        GEO VARCHAR(20) NOT NULL,
                                        PRICE_PRERMT FLOAT(6) NOT NULL,
                                        PRIMARY KEY (CD_ID)
                                 )")
daily_fx <- dbExecute(connect, "CREATE TABLE DAILY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL,
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )")
monthly_fx <- dbExecute(connect, "CREATE TABLE MONTHLY_FX(
                                  DFX_ID INTEGER NOT NULL,
                                  DATE DATE NOT NULL,
                                  FXUSDCAD FLOAT(6) NOT NULL,
                                  PRIMARY KEY (DFX_ID))")

### Verify the table in database 

In [5]:
if(crop_data == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(connect)
  print (msg)
}else {
  cat ("Table was created successfully.\n")
}
if(farm_prices == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(connect)
  print (msg)
}else {
  cat ("Table was created successfully.\n")
}
if(daily_fx == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(connect)
  print (msg)
}else {
  cat ("Table was created successfully.\n")
}
if(monthly_fx == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(connect)
  print (msg)
}else {
  cat ("Table was created successfully.\n")
}
dbListTables(connect)

Table was created successfully.


Table was created successfully.


Table was created successfully.


Table was created successfully.


[1] "CROP_DATA"   "DAILY_FX"    "FARM_PRICES" "MONTHLY_FX"

### Pull datasets and load tables into database 

In [6]:
crop_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Practice%20Assignment/Annual_Crop_Data.csv', colClasses=c(YEAR="character"))
farm_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv', colClasses = c(DATE= 'character'))
daily_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Practice%20Assignment/Daily_FX.csv', colClasses=c(DATE="character"))
month_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv', colClasses = c(DATE = 'character'))


### Inspect each datasets

In [7]:
head(crop_df)
head(farm_df)
head(daily_df)
head(month_df)

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<chr>,<chr>,<chr>,<dbl>
1,0,1985-01-01,Barley,Alberta,127.39
2,1,1985-01-01,Barley,Saskatchewan,121.38
3,2,1985-01-01,Canola,Alberta,342.00
4,3,1985-01-01,Canola,Saskatchewan,339.82
5,4,1985-01-01,Rye,Alberta,100.77
6,5,1985-01-01,Rye,Saskatchewan,109.75


,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-01,1.319276
2,1,2017-02-01,1.310726
3,2,2017-03-01,1.338643
4,3,2017-04-01,1.344021
5,4,2017-05-01,1.360705
6,5,2017-06-01,1.329805


### load into database

In [8]:
dbWriteTable(connect, "CROP_DATA", crop_df, overwrite=TRUE, header = TRUE)
dbWriteTable(connect, "FARM_PRICES", farm_df, overwrite=TRUE, header = TRUE)
dbWriteTable(connect, "DAILY_FX", daily_df, overwrite=TRUE, header = TRUE)
dbWriteTable(connect, "MONTHLY_FX", month_df, overwrite=TRUE, header = TRUE)
dbListTables(connect)

[1] "CROP_DATA"   "DAILY_FX"    "FARM_PRICES" "MONTHLY_FX"

### Now we have database and tables. it's the time we use sql to solve the problems

#### There are 10 questions we need to answer: 
1. How many records are in the farm prices dataset?
2. Which geographies are included in the farm prices dataset?
3. How many hectares of Rye were harvested in Canada in 1968?
4. Query and display the first 6 rows of the farm prices table for Rye.
5. Which provinces grow Barley?
6. Find the first and last dates for the farm prices data.
7. Which crops have ever reached a farm price greater than or equal to $350 per metric tonne?
8. Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best
9. Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000?
10. Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.
11. Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.

##### Question 1: How many records are in the farm prices dataset?

In [9]:
dbGetQuery(connect, "SELECT COUNT(CD_ID) AS Number_of_Records FROM FARM_PRICES")

Number_of_Records
<int>
2678


##### Question 2: Which geographies are included in the farm prices dataset?

In [10]:
dbGetQuery(connect, "SELECT DISTINCT GEO AS Geopraphies FROM FARM_PRICES")

Geopraphies
<chr>
Alberta
Saskatchewan


##### Question 3:  How many hectares of Rye were harvested in Canada in 1968?

In [11]:
dbGetQuery(connect, "SELECT COUNT(*) AS Hectares_of_Rye_Canada1968 FROM CROP_DATA 
                     WHERE strftime('%Y', YEAR) = '1968' AND 
                     GEO= 'Canada' AND CROP_TYPE = 'Rye'")

Hectares_of_Rye_Canada1968
<int>
1


##### Question 4: Query and display the first 6 rows of the farm prices table for Rye.

In [12]:
dbGetQuery(connect, "SELECT * FROM FARM_PRICES WHERE CROP_TYPE = 'Rye' LIMIT 6")

CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
<int>,<chr>,<chr>,<chr>,<dbl>
4,1985-01-01,Rye,Alberta,100.77
5,1985-01-01,Rye,Saskatchewan,109.75
10,1985-02-01,Rye,Alberta,95.05
11,1985-02-01,Rye,Saskatchewan,103.46
16,1985-03-01,Rye,Alberta,96.77
17,1985-03-01,Rye,Saskatchewan,106.38


##### Question 5: Which provinces grew Barley?

In [13]:
# Exclude Canada because we only want provinces

dbGetQuery(connect, "SELECT DISTINCT GEO AS PROVINCES_GREW_BARLEY FROM CROP_DATA 
                     WHERE CROP_TYPE = 'Barley' AND NOT GEO = 'Canada' ")


PROVINCES_GREW_BARLEY
<chr>
Alberta
Saskatchewan


##### Question 6: Find the first and last dates for the farm prices data.

In [14]:
dbGetQuery(connect, "SELECT MIN(DATE) AS FIRST_DATE, MAX(DATE) AS LAST_DATE FROM FARM_PRICES")

FIRST_DATE,LAST_DATE
<chr>,<chr>
1985-01-01,2020-12-01


##### Question 7: Which crops have ever reached a farm price greater than or equal to $350 per metric tonne?

In [15]:
dbGetQuery(connect, "SELECT DISTINCT CROP_TYPE FROM FARM_PRICES WHERE PRICE_PRERMT >= 350")

CROP_TYPE
<chr>
Canola


##### Question 8: Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best?

In [16]:
dbGetQuery(connect, "SELECT CROP_TYPE, strftime('%Y', YEAR) AS YEAR, AVG_YIELD  FROM CROP_DATA
                     WHERE GEO = 'Saskatchewan' AND 
                     strftime('%Y', YEAR) = '2000' 
                     ORDER BY AVG_YIELD DESC
           ")

CROP_TYPE,YEAR,AVG_YIELD
<chr>,<chr>,<int>
Barley,2000,2800
Wheat,2000,2200
Rye,2000,2100
Canola,2000,1400


##### Question 9: Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000?

In [17]:
dbGetQuery(connect, "SELECT strftime('%Y', YEAR) AS YEAR, CROP_TYPE, GEO, AVG_YIELD
                     FROM CROP_DATA
                     WHERE YEAR >= 2000
                     ORDER BY AVG_YIELD DESC
                     LIMIT 10 
           ")

YEAR,CROP_TYPE,GEO,AVG_YIELD
<chr>,<chr>,<chr>,<int>
2013,Barley,Alberta,4100
2016,Barley,Alberta,4100
2020,Barley,Alberta,3980
2013,Wheat,Alberta,3900
2016,Barley,Canada,3900
2016,Wheat,Alberta,3900
2017,Barley,Alberta,3900
2019,Barley,Alberta,3890
2020,Barley,Canada,3820


##### Question 10: Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.

In [18]:
dbGetQuery(connect, "SELECT MAX(YEAR) AS RECENT_YEAR, SUM(HARVESTED_AREA) AS Total_HARVESTED 
                     FROM CROP_DATA
                     WHERE strftime('%Y', YEAR) = (SELECT MAX(strftime('%Y', YEAR)) 
                                                   FROM CROP_DATA WHERE GEO = 'Canada' AND 
                                                   CROP_TYPE = 'Wheat')
           ")

RECENT_YEAR,Total_HARVESTED
<chr>,<int>
2020-12-31,38897100


##### Question 11: Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.

In [19]:
dbGetQuery(connect, "SELECT F.DATE, F.CROP_TYPE, F.GEO, F.PRICE_PRERMT AS CANADIAN_CURRENCY, 
                     (F.PRICE_PRERMT / M.FXUSDCAD) AS US_CURRENCY
                     FROM FARM_PRICES F, MONTHLY_FX M
                     WHERE F.DATE = M.DATE AND
                     CROP_TYPE = 'Canola' AND
                     GEO = 'Saskatchewan'
                     ORDER BY F.DATE DESC
                     LIMIT 6
                
           
           ")

DATE,CROP_TYPE,GEO,CANADIAN_CURRENCY,US_CURRENCY
<chr>,<chr>,<chr>,<dbl>,<dbl>
2020-12-01,Canola,Saskatchewan,507.33,396.1128
2020-11-01,Canola,Saskatchewan,495.64,379.2718
2020-10-01,Canola,Saskatchewan,474.80,359.2965
2020-09-01,Canola,Saskatchewan,463.52,350.4057
2020-08-01,Canola,Saskatchewan,464.60,351.3827
2020-07-01,Canola,Saskatchewan,462.88,342.9122
